In [1]:
pip install falcon

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\12162\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from wsgiref.simple_server import make_server
import falcon

# Falcon follows the REST architectural style, meaning (among
# other things) that you think in terms of resources and state
# transitions, which map to HTTP verbs.
class ClassifyResourceV1:
    def on_get(self, req, resp):
        # Gets the img query parameter
        img_url = req.get_param('img')
        # Creates a result dictionary with a single entry
        result = dict(src_img=img_url)
        # Sets the response content to the result dictionary
        resp.media = result
        """Handles GET requests"""
        resp.status = falcon.HTTP_200  # This is the default status

        
# falcon.App instances are callable WSGI apps
# in larger applications the app is created in a separate file
app_v1 = falcon.App()

# Resources are represented by long-lived class instances
classify_v1 = ClassifyResourceV1()

# things will handle all requests to the '/classify' URL path
app_v1.add_route('/classify', classify_v1)

In [3]:
def run_server(app):
    with make_server('', 9999, app) as httpd:
        print('Serving on port 9999...')

        # Serve until process is killed
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Stopping web server')


In [ ]:
run_server(app_v1)

Serving on port 9999...


127.0.0.1 - - [04/Mar/2023 15:17:57] "GET /classify?img=https%3A%2F%2Fstardewvalleywiki.com%2Fmediawiki%2Fimages%2Fa%2Fa4%2FBanner_Right_Winter.png HTTP/1.1" 200 90
127.0.0.1 - - [04/Mar/2023 15:18:09] "GET /classify?img=https%3A%2F%2Fwww.stardewvalley.net%2Fwp-content%2Fuploads%2F2018%2F12%2F1_1screenshot15.png HTTP/1.1" 200 91


In [ ]:
from pathlib import Path
import tempfile

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np
import requests

import json

# Downloads and loads the ResNetModel
model = ResNet50(weights='imagenet')

In [ ]:
def load_img_url(img_url, target_size):
    r = requests.get(img_url)
    if r.status_code != 200:
        print('Could not download image')
        return None
    
    img_suffix = '.{}'.format(img_url.split('.')[-1])
    
    with tempfile.NamedTemporaryFile(suffix=img_suffix) as fp:
        fp.write(r.content)
        tf_path = Path(fp.name)
        img = image.load_img(tf_path, target_size=target_size)
    
    return img

In [ ]:
def make_prediction(model, img_url, target_size=(224, 224), top=5):
    result = dict(src_img=img_url)
    img = load_img_url(img_url, target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    
    result['predictions'] = []

    for class_name, class_description, score in decode_predictions(preds, top=top)[0]:
        result['predictions'].append(dict(
            class_name=str(class_name),
            class_description=str(class_description),
            score=float(score)
        ))
        
    return result   

In [ ]:
# TODO: Test the `make_prediction` function using img_urls

# TODO: Add img_urls for testing
img_urls = []

for img_url in img_urls:
    result = make_prediction(model, img_url)
    print('-----------------------------')
    print('')
    # Prints a pretty version of the result
    print(json.dumps(result, indent=2))
    print('-----------------------------')
    print()

In [ ]:
class ClassifyResourceV2:
    def on_get(self, req, resp):
        # TODO: Implement the code to handle the GET requests

app_v2 = falcon.App()

classify_v1 = ClassifyResourceV2()

app_v2.add_route('/classify', classify_v2)

In [ ]:
run_server(app_v2)